In [75]:
import sys
sys.path.append("..")
import imagenet_load
from importlib import reload
reload(imagenet_load)
import tarfile
import numpy as np
import imread
%pylab inline
import imread
from imread import imread_from_blob
from imagenet_load import orient
import multigpu
reload(multigpu)
import filter_gen
import conv
reload(conv)
import gc
import logging
import dill
from scipy.misc import imresize
import opt

Populating the interactive namespace from numpy and matplotlib


/data/vaishaal/anaconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['imread']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [49]:
%time loader = imagenet_load.ImagenetLoader(1000, classes_path="../classes", mmap_loc="/dev/shm/imagenet_train", n_procs=32, preprocess=preprocess, image_shape=(3, 64, 64))

%time loader_val = imagenet_load.ImagenetLoader(1000, classes_path="../classes", tarball_func=imagenet_load.VAL_LOADER, mmap_loc="/dev/shm/imagenet_val", n_procs=32, preprocess=preprocess, image_shape=(3, 64, 64))

There exists a matrix at /dev/shm/imagenet_train


CPU times: user 232 ms, sys: 348 ms, total: 580 ms
Wall time: 14.4 s


There exists a matrix at /dev/shm/imagenet_val


CPU times: user 208 ms, sys: 300 ms, total: 508 ms
Wall time: 903 ms


In [39]:
def preprocess(im):
    return imresize(im, size=(64,64,3))

In [ ]:
%time X_train = loader.load_all()
%time X_val = loader_val.load_all()

Loaded 0 classes into mem
Loaded 100 classes into mem
Loaded 200 classes into mem
Loaded 300 classes into mem


In [73]:
X_train = X_train.reshape(X_train.shape[0], -1).astype('float64')
X_val = X_val.reshape(X_val.shape[0], -1).astype('float64')

In [ ]:
Y = np.eye(1000)[loader.Y.astype('int')]

In [ ]:
%time XtX = X_train.T.dot(X_train)
%time Xty = X_train.T.dot(Y)

In [ ]:
def run_experiment_linear_64_64